In [1]:
import os, re  # 파일경로 및 처리, 정규표현식사용(search,findall,macht등)
import numpy as np #대규모 수학연산
import glob
import tensorflow as tf #데이터의 배열,딥러닝 구현 (뉴런생성,훈련등)
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [2]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()  #1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)#2
    sentence = re.sub(r'[" "]+', " ", sentence)#3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)#4
    sentence = sentence.strip()#5
    sentence = '<start> ' + sentence + ' <end>'#6
    
    if len(sentence.split()) > 15: #총길이가 15를 넘으면 공백으로 바꿔준다.
        sentence = ""
        
    return sentence
# 정규표현식이 파라미터가 무엇이 들어가고 어떠한 역할을하나




In [3]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if len(sentence.split()) >= 12: continue   
    if sentence[-1] == ":": continue
        
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    


# 데이터 토큰화 함수
def tokenize(corpus):
    # 15000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 15000단어에 포함되지 못한 단어는 '<unk>' 로 바꿀거에요
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길으를 맞추고 싶다면 padding='pre'를 사용합니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   51    5 ...    0    0    0]
 [   2   17 3146 ...    0    0    0]
 [   2   34    7 ...   43    3    0]
 ...
 [   2    5  120 ...    0    0    0]
 [   2  256  196 ...   12    3    0]
 [   2    7   35 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f426facd070>


In [4]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

#train ,val 데이터 값 split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True,
                                                          random_state=34)
#분류된 데이터 확인
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)


Source Train: (121675, 14)
Target Train: (121675, 14)


In [5]:
#모델 생성
BUFFER_SIZE = len(src_input) #입력값의 길이 크기의 버퍼
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1 #생성한 사전의 단어 15000개와 <pad>포한한 15000+1개의 크기

#데이터 소스로부터 데이터셋 생성
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)) 
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [6]:
###########인공지능 학습시키기
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-6.90162415e-05, -9.29166927e-05, -1.21971680e-05, ...,
         -3.68250858e-06,  5.93190780e-05,  2.75775237e-04],
        [ 5.43765163e-05, -1.08063628e-04,  3.39067745e-04, ...,
         -1.19736018e-04,  5.33084203e-05,  2.70043791e-04],
        [-7.25257269e-06,  6.01785223e-06,  7.15591421e-04, ...,
         -3.56419972e-04,  1.41216267e-04,  4.07147076e-04],
        ...,
        [-1.43505586e-03,  2.96873366e-03,  3.52545059e-04, ...,
         -7.85973272e-04, -6.14626682e-04, -1.89613434e-04],
        [-1.52850687e-03,  3.15520819e-03,  1.77560098e-04, ...,
         -7.84791657e-04, -6.80675672e-04, -1.88082267e-04],
        [-1.58217666e-03,  3.30188544e-03, -2.98748978e-06, ...,
         -7.98921450e-04, -7.45906553e-04, -1.80574338e-04]],

       [[-6.90162415e-05, -9.29166927e-05, -1.21971680e-05, ...,
         -3.68250858e-06,  5.93190780e-05,  2.75775237e-04],
        [-2.10255792e-04,  1.09952241e-04, -3

In [7]:
# optimizer와 loss
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
594/594 [==============================] - 95s 153ms/step - loss: 3.3416
Epoch 2/10
594/594 [==============================] - 99s 167ms/step - loss: 2.8613
Epoch 3/10
594/594 [==============================] - 105s 177ms/step - loss: 2.6928
Epoch 4/10
594/594 [==============================] - 108s 182ms/step - loss: 2.5651
Epoch 5/10
594/594 [==============================] - 108s 182ms/step - loss: 2.4525
Epoch 6/10
594/594 [==============================] - 108s 182ms/step - loss: 2.3484
Epoch 7/10
594/594 [==============================] - 108s 182ms/step - loss: 2.2517
Epoch 8/10
594/594 [==============================] - 108s 182ms/step - loss: 2.1608
Epoch 9/10
594/594 [==============================] - 108s 182ms/step - loss: 2.0752
Epoch 10/10
594/594 [==============================] - 108s 182ms/step - loss: 1.9944


In [8]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다.
    while True:
        predict = model(test_tensor) # 입력받은 문장의 텐서를 입력합니다.
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다.
        
        # 우리 모델이 새롭계 예측한 단어를 입력 문장의 뒤에 붙여줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면 while 루프를 돌면서 다음 단어를 예측해야합니다.
        if predict_word.numpy()[0] == end_token:
            break
        if test_tensor.shape[1] >= max_len:
            break
    
    generated = ""
    
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다.
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    
    return generated # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [9]:
generate_text(model, tokenizer, init_sentence="<start> love",  max_len=20)

'<start> love is a beautiful thing <end> '